In [3]:
# updated on 24.06.16
# 지역 정보를 기본으로 주택유형, 매매유형 등에 따라 조회후 저장 (저정은 한번에 또는 code별로 가능)


import requests
import time
import random
import folium
import webbrowser
import pandas as pd
from datetime import datetime

MaxPages = 3000

def load_address_codes(file_path):
    address_codes = {}
    with open(file_path, 'r', encoding='cp949') as f:  # ANSI로 저장된 경우 cp949 인코딩 사용
        for line in f:
            code, name, status = line.strip().split('\t')
            if status == '존재':  # 폐지된 법정동은 무시
                address_codes[code] = name
    return address_codes
    
    return address_codes

def search_properties_by_condition(region_code, house_type, transaction_type, address_codes, dprc_Max=5000000, dprc_Min = 1000):
    # url = f"https://m.land.naver.com/cluster/ajax/articleList?rletTpCd={house_type}&tradTpCd={transaction_type}&cortarNo={region_code}&showR0&page=1"
    url = f"https://m.land.naver.com/cluster/ajax/articleList?rletTpCd={house_type}&tradTpCd={transaction_type}&cortarNo={region_code}&dprcMax={dprc_Max}&dprcMin={dprc_Min}&showR0&page=1"

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36",
        "Referer": "https://m.land.naver.com/map/"
    }

    all_data = []

    for attempt in range(MaxPages):
        try:
            print(f">> {url}")
            response = requests.get(url, headers=headers)
            response.raise_for_status()
            data = response.json()

            if not data.get('body'):
                print("더 이상 정보가 없습니다.")
                break

            # 현재 페이지의 데이터를 all_data 리스트에 추가
            for property_info in data['body']:
                cortarNo = property_info.get('cortarNo', '')
                if cortarNo in address_codes:  # 주어진 코드가 address_codes에 있을 때만 추가
                    house_info = {
                        '법정동명': address_codes[cortarNo],
                        '주택 유형': property_info.get('rletTpNm', ''),
                        '거래 유형': property_info.get('tradTpNm', ''),
                        '아파트명': property_info.get('atclNm', ''),
                        '가격': int(str(property_info.get('prc', '')).replace(',', '')),  # 가격을 문자열로 변환 후 쉼표 제거하여 정수로 변환
                        '면적(spc1)': property_info.get('spc1', ''),
                        '면적(spc2)': property_info.get('spc2', ''),
                        '특징 설명': property_info.get('atclFetrDesc', ''),
                        '동호수': property_info.get('buidNm', ''),
                        '층': property_info.get('flrInfo', ''),
                        '방향': property_info.get('direction', ''),
                        '게시일': property_info.get('atclCfmYmd', ''),
                        '위도': property_info.get('lat', ''),
                        '경도': property_info.get('lng', ''),
                        '업체': property_info.get('rltrNm', ''),  # 업체 추가
                    }
                    all_data.append(house_info)

            # 추가 페이지를 요청하기 위해 URL 변경
            url_split = url.split("&page=")
            page_number = int(url_split[1])
            next_page_url = url_split[0] + "&page=" + str(page_number + 1)
            url = next_page_url

        except requests.exceptions.RequestException as e:
            print(f"HTTP 요청 오류 발생: {str(e)} (시도 {attempt + 1}/5)")
            time.sleep(2 ** attempt)
        except ValueError:
            print("JSON 디코딩 오류 발생")
            break
        retry_interval = random.uniform(1.1, 5.5) 
        time.sleep(retry_interval)

    # 가격을 기준으로 오름차순으로 정렬
    all_data_sorted = sorted(all_data, key=lambda x: x['가격'])

    return all_data_sorted

def calculate_center_of_coordinates(coordinates):
    """
    좌표들의 평균값을 계산하여 반환합니다.
    :param coordinates: 좌표들의 리스트. 각 좌표는 [위도, 경도] 형태여야 합니다.
    :return: 평균 좌표 [평균 위도, 평균 경도]
    """
    total_lat = sum(coord[0] for coord in coordinates)
    total_lng = sum(coord[1] for coord in coordinates)
    avg_lat = total_lat / len(coordinates)
    avg_lng = total_lng / len(coordinates)
    return [avg_lat, avg_lng]

def save_properties_to_excel(properties, file_name):
    # DataFrame 생성
    df = pd.DataFrame(properties)

    # 엑셀 파일로 저장
    df.to_excel(file_name, index=False, sheet_name='sheet1')

    # Pivot 테이블 생성
    pivot_table = df.groupby(['법정동명', '아파트명', '면적(spc1)'])['가격'].agg(['max', 'min', 'mean', 'count']).reset_index()
    
    # 계산 결과를 정수로 변환
    pivot_table['max'] = pivot_table['max'].astype(int)
    pivot_table['min'] = pivot_table['min'].astype(int)
    pivot_table['mean'] = pivot_table['mean'].astype(int)
    pivot_table['count'] = pivot_table['count'].astype(int)
    pivot_table['평균(면적/3.3)'] = (pivot_table['mean'] / (pivot_table['면적(spc1)'].astype(float) / 3.3)).astype(int)
    
    # 엑셀 파일에 Pivot 테이블 추가
    with pd.ExcelWriter(file_name, engine='openpyxl', mode='a') as writer:
        pivot_table.to_excel(writer, sheet_name='sheet2', index=False)

if __name__ == "__main__":
    region_codes = {
    
        # "강동구": "1174000000",
        # "강남구": "1168000000",
        # "서초구": "1165000000",
        # "송파구": "1171000000",
        # "동작구": "1159000000",
        # "영등포구": "1156000000",
        # "양천구": "1147000000",
        # "마포구": "1144000000",
        # "용산구": "1117000000",
        # "성동구": "1120000000",
        # "광진구": "1121500000",
        # "노원구": "1135000000",
        # "도봉구": "1132000000",
        # "강북구": "1130000000",
        # "동대문구": "1123000000",
        # "중랑구": "1126000000",
        
        # "하남시": "4145000000",
        # "구리시": "4131000000",
        # "분당구": "4113500000",
        
        # "연수구": "2818500000",
        # "인천서구": "2826000000",
        # "영통구": "4111700000",	
        # "수지구":"4146500000",
        # "기흥구": "4146300000",
        # "화성시": "4159000000", 
        
        # "세종시": "3611000000", 
        # "유성구": "3020000000",
        # "천안시서북": "4413300000",
        # "아산시": "4420000000",
        # "달서구": "2729000000",    

        # "수성구": "2726000000",
        # "해운대구": "2635000000",
    
        # "광주시경기": "4161000000",
        # "남양주시": "4136000000",    
        # "양평군": "4183000000",
        # "가평군": "4182000000",
        # "남양주시": "4136000000",
        # "여주시": "4167000000",
        "의정부시": "4115000000",
        # "원주시": "5113000000",
        # "제주시": "5011000000",
        # "서귀포시": "5013000000",
    
        # "속초시": "5121000000", 
        # "강릉시": "5115000000",	
        # "양양군": "5183000000",
        # "경산시": "4729000000",
        # "청주시 흥덕구": "4311300000",     
        # "음성군": "4377000000",
        # "증평군": "4374500000",
        # "진천군": "4375000000",
        # "괴산군": "4376000000",
        # "의성군": "4773000000",

        # "암사동": "1174010700",
        # "괴산군": "4376000000",
        # "둔촌동": "1174010600",
        # "청담동": "1168010400",
        # "개포동": "1168010300",
        # "대치동": "1168010600",
        # "삼성동": "1168010500",
        # "잠실동": "1171010100",
        # "신천동": "1171010200",
        # 다른 지역 코드도 필요에 따라 추가 가능
    }

    
    # 검색 조건, 주택유형, 거래유형, 최대매매가, 최소매매가
    house_type = "APT:JGC:ABYG"  # 주택 유형, 없는 키워드면 전체, "APT:JGC:ABYG"
    transaction_type = "A1"  # 거래 유형   A1 매매 B1 전세 B2 월세
    dprc_Max = 5000000            # 최대 매매 가격 10000 : 1억원
    dprc_Min = 1000            # 최대 매매 가격 10000 : 1억원

    # address_code.txt 파일에서 법정동 코드와 법정동명을 읽어옴
    address_codes = load_address_codes('address_code.txt')

    all_properties = []

    for region_name, region_code in region_codes.items():
        region_properties = search_properties_by_condition(region_code, house_type, transaction_type, address_codes, dprc_Max, dprc_Min)
        # all_properties.extend(region_properties)

        # 개별 지역별 파일로 저장
        now = datetime.now()
        timestamp = now.strftime("%Y%m%d_%H%M%S")
        excel_file_name = f"{region_name}_properties_{timestamp}.xlsx"
        save_properties_to_excel(region_properties, excel_file_name)

    # # 전체 데이터를 하나의 파일로 저장
    # now = datetime.now()
    # timestamp = now.strftime("%Y%m%d_%H%M%S")
    # excel_file_name_all = f"전체_properties_{timestamp}.xlsx"
    # save_properties_to_excel(all_properties, excel_file_name_all)

    print('Finished')

>> https://m.land.naver.com/cluster/ajax/articleList?rletTpCd=APT:JGC:ABYG&tradTpCd=A1&cortarNo=4115000000&dprcMax=5000000&dprcMin=1000&showR0&page=1
>> https://m.land.naver.com/cluster/ajax/articleList?rletTpCd=APT:JGC:ABYG&tradTpCd=A1&cortarNo=4115000000&dprcMax=5000000&dprcMin=1000&showR0&page=2
>> https://m.land.naver.com/cluster/ajax/articleList?rletTpCd=APT:JGC:ABYG&tradTpCd=A1&cortarNo=4115000000&dprcMax=5000000&dprcMin=1000&showR0&page=3
>> https://m.land.naver.com/cluster/ajax/articleList?rletTpCd=APT:JGC:ABYG&tradTpCd=A1&cortarNo=4115000000&dprcMax=5000000&dprcMin=1000&showR0&page=4
>> https://m.land.naver.com/cluster/ajax/articleList?rletTpCd=APT:JGC:ABYG&tradTpCd=A1&cortarNo=4115000000&dprcMax=5000000&dprcMin=1000&showR0&page=5
>> https://m.land.naver.com/cluster/ajax/articleList?rletTpCd=APT:JGC:ABYG&tradTpCd=A1&cortarNo=4115000000&dprcMax=5000000&dprcMin=1000&showR0&page=6
>> https://m.land.naver.com/cluster/ajax/articleList?rletTpCd=APT:JGC:ABYG&tradTpCd=A1&cortarNo=4115